In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import plotly.express as px
import math
import gc
import seaborn as sns
import lightgbm as lgbm 
import xgboost as xgb

from tqdm import tqdm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler,PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV


In [2]:
df = pd.read_csv('./data/df_stat.csv')

from sklearn.metrics import mean_squared_error
def metric(y,x):
    return round(np.sqrt(mean_squared_error(x,y)),3)

df = pd.get_dummies(df, columns=['location'], drop_first=True)

RMSE_train = []
RMSE_test = []

feat = ['mean_atmos_press',
 'median_wind_spd',
 'mean_wind_dir',
 'mean_wind_spd',
 'mean_temp',
 'mean_rel_humidity',
 'std_wind_dir',
 'median_atmos_press',
 'var_wind_dir',
 'ptp_wind_spd',
 'median_wind_dir',
 'std_rel_humidity',
 'std_atmos_press',
 'var_rel_humidity',
 'var_atmos_press',
 'std_wind_spd',
 'var_temp',
 'var_wind_spd',
 'std_temp',
 'min_atmos_press',
 'mean_precip',
 'median_rel_humidity',
 'median_temp',
 'min_temp',
 'std_precip',
 'ptp_wind_dir',
 'var_precip',
 'ptp_atmos_press',
 'max_atmos_press',
 'min_wind_spd',
 'max_rel_humidity',
 'min_wind_dir',
 'ptp_temp',
 'ptp_rel_humidity',
 'max_wind_spd',
 'max_wind_dir',
 'max_temp',
 'min_rel_humidity',
 'max_precip',
 'ptp_precip',
 'location_E',
 'location_B',
 'location_D',
 'location_C',
 'median_precip',
 'min_precip']
RSEED= 42

In [3]:
X = df[feat]
y = df.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = RSEED)

columns = X_train.columns

In [5]:
scalerStand = StandardScaler()
X_train = scalerStand.fit_transform(X_train)
X_test = scalerStand.transform(X_test)

# scaled X_train data to pandas Dataframe
X_train = pd.DataFrame(X_train)
X_train.columns = columns
#X_train.drop('Unnamed: 0', axis=1, inplace=True)

# scaled X_test data to pandas Dataframe
X_test = pd.DataFrame(X_test)
X_test.columns = columns

#X_test.drop('Unnamed: 0', axis=1, inplace=True)

/Users/jonigul/neuefische/AirQo-Ugandan-Air-Quality-Forecast-Challenge/.venv/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


,mean_atmos_press,median_wind_spd,mean_wind_dir,mean_wind_spd,mean_temp,mean_rel_humidity,std_wind_dir,median_atmos_press,var_wind_dir,ptp_wind_spd,...,max_temp,min_rel_humidity,max_precip,ptp_precip,location_E,location_B,location_D,location_C,median_precip,min_precip
0,-1.212648,-0.260316,1.328238,-0.401591,0.877026,-1.936137,-0.309207,-1.214447,-0.393413,-0.744315,...,0.410589,-1.357275,-0.822808,-0.822808,-0.480281,-0.22853,-0.693752,2.813806,0.0,0.0
1,1.455926,-0.186745,-0.546123,-0.157651,0.780520,-0.919442,-0.253867,1.460506,-0.341562,0.002962,...,0.723994,-1.361587,0.077746,0.077746,-0.480281,-0.22853,1.441437,-0.355391,0.0,0.0
2,-1.154626,-0.309809,2.003618,-0.410143,-1.746480,0.712793,-0.349213,-1.164397,-0.430530,-0.436237,...,-1.094530,0.030928,0.104596,0.104596,-0.480281,-0.22853,-0.693752,2.813806,0.0,0.0
3,-1.047987,-0.307134,0.649684,-0.381794,-1.235970,0.885622,-0.672345,-1.063528,-0.719046,-0.465019,...,-1.195130,0.779592,0.835931,0.835931,-0.480281,-0.22853,-0.693752,2.813806,0.0,0.0
4,1.334258,-0.173368,-0.484815,-0.168409,0.442225,-0.747799,0.176778,1.347318,0.082058,0.032810,...,0.619526,-1.029588,-0.857778,-0.857778,-0.480281,-0.22853,1.441437,-0.355391,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3103,-0.656623,-0.104479,0.301349,-0.054098,-1.333992,1.643082,1.606040,-0.645422,1.743602,-0.200114,...,-1.195130,1.538447,0.244584,0.244584,-0.480281,-0.22853,-0.693752,-0.355391,0.0,0.0
3104,-0.540098,-0.537880,0.176627,-0.083065,0.367979,-0.384906,1.641662,-0.531848,1.790031,-0.573220,...,-1.357636,0.951197,-0.855937,-0.855937,-0.480281,-0.22853,-0.693752,-0.355391,0.0,0.0
3105,1.465480,-0.324523,-1.227701,-0.065476,0.483107,-0.194537,-0.069135,1.458966,-0.164211,0.683080,...,0.139745,0.091449,-0.496495,-0.496495,-0.480281,-0.22853,1.441437,-0.355391,0.0,0.0
3106,-0.777498,-0.386056,0.610338,-0.300452,-0.649352,1.434158,1.437764,-0.775551,1.527575,-0.347224,...,-0.448374,1.159882,0.003260,0.003260,-0.480281,-0.22853,-0.693752,-0.355391,0.0,0.0


## Random Forest Regressor

In [ ]:
rfr = RandomForestRegressor()
rfr = rfr.fit(X_train, y_train)

# predict on test-set
y_pred_rfr = rfr.predict(X_test)
y_pred_train_rfr = rfr.predict(X_train)

# RMSE on train and test set
print("Train RMSE:", metric(y_train, y_pred_train_rfr))
print("Test RMSE:", metric(y_test, y_pred_rfr))
RMSE_train.append(metric(y_train, y_pred_train_rfr))
RMSE_test.append(metric(y_test, y_pred_rfr))

In [ ]:
importances = rfr.feature_importances_
forest_importances = pd.Series(importances)
fi = pd.DataFrame(forest_importances)
fi = fi.set_index(columns[1:])
fi.plot.bar(figsize=(15,10))

In [ ]:
# Hyperparameter grid
param_grid = {
    'n_estimators': np.linspace(10, 200).astype(int),
    'max_depth': [None] + list(np.linspace(3, 20).astype(int)),
    'max_features': ['auto', 'sqrt', None] + list(np.arange(0.5, 1, 0.1)),
    'max_leaf_nodes': [None] + list(np.linspace(10, 50, 500).astype(int)),
    'min_samples_split': [2, 5, 10],
    'bootstrap': [True, False]
}

# Create the random search model
gs = GridSearchCV(rfr, param_grid, n_jobs = -1, 
                        scoring = 'roc_auc', cv = 3, 
                        verbose = 5)

# Fit 
gs.fit(X_train, y_train)

    

## KNN

In [ ]:
knn = KNeighborsRegressor()
knn = knn.fit(X_train, y_train)
# predict on test-set
y_pred_knn = knn.predict(X_test)
y_pred_train_knn = knn.predict(X_train)

# RMSE on train and test set
print("Train RMSE:", metric(y_train, y_pred_train_knn))
print("Test RMSE:", metric(y_test, y_pred_knn))
RMSE_train.append(metric(y_train, y_pred_train_knn))
RMSE_test.append(metric(y_test, y_pred_knn))

## Plot different RMSE

In [ ]:
RMSE = {'RMSE_train': RMSE_train, 'RMSE_test': RMSE_test, 'Model':['Random Forest','KNN']}
baseline = pd.DataFrame(RMSE)
baseline = baseline.set_index('Model')
baseline.plot.bar()